<a href="https://colab.research.google.com/github/sanchianatisha/TaskWeek7/blob/main/TaskWeek7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Nama** : Sanchia Natisha Kenzie
### **NPM** : 2106724990


In [1]:
pip install transformers torch

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# The correct model name is 'distilbert-base-uncased-finetuned-sst-2-english'
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [3]:
sentence = "This is a great movie!"
inputs = tokenizer(sentence, return_tensors="pt")

In [4]:
import torch

In [5]:
outputs = model(**inputs)
predicted_label = torch.argmax(outputs.logits, dim=-1).item()

In [6]:
import random

# Define vocabulary with positive and negative words
positive_words = ["great", "amazing", "wonderful", "happy", "excellent"]
negative_words = ["bad", "terrible", "awful", "sad", "horrible"]

# Create a function to generate a sentence
def generate_sentence(sentiment):
    sentence = []
    for _ in range(random.randint(5, 10)):
        if sentiment == 1:
            sentence.append(random.choice(positive_words))
        else:
            sentence.append(random.choice(negative_words))
    return " ".join(sentence)

# Generate synthetic dataset
synthetic_dataset = []
for _ in range(1000):
    sentiment = random.randint(0, 1)
    sentence = generate_sentence(sentiment)
    synthetic_dataset.append({"text": sentence, "label": sentiment})

print(synthetic_dataset)

[{'text': 'sad terrible horrible horrible awful terrible', 'label': 0}, {'text': 'awful awful awful terrible sad sad', 'label': 0}, {'text': 'excellent happy happy wonderful amazing excellent great excellent happy happy', 'label': 1}, {'text': 'great happy great excellent wonderful amazing wonderful happy great', 'label': 1}, {'text': 'great wonderful happy wonderful happy', 'label': 1}, {'text': 'sad terrible bad terrible sad awful sad horrible bad awful', 'label': 0}, {'text': 'happy amazing excellent excellent great amazing wonderful happy great', 'label': 1}, {'text': 'happy excellent amazing great amazing amazing happy amazing excellent', 'label': 1}, {'text': 'excellent happy great great great great excellent happy great amazing', 'label': 1}, {'text': 'excellent excellent excellent great happy', 'label': 1}, {'text': 'sad horrible bad sad terrible terrible sad awful terrible horrible', 'label': 0}, {'text': 'terrible horrible terrible awful horrible sad horrible horrible sad', '

In [7]:
from sklearn.metrics import accuracy_score

predictions = []
true_labels = []

for data in synthetic_dataset:
    inputs = tokenizer(data["text"], return_tensors="pt")
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=-1).item()

    predictions.append(predicted_label)
    true_labels.append(data["label"])

accuracy = accuracy_score(true_labels, predictions)
print("Accuracy before attention:", accuracy)

Accuracy before attention: 1.0


In [12]:
from transformers import DistilBertForSequenceClassification, DistilBertConfig

# Assuming your model is a DistilBERT-based model
class AttentionBERT(DistilBertForSequenceClassification):
    def __init__(self, config):
        super().__init__(config)

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Get the model's hidden states
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask, labels=labels, output_hidden_states=True)

        # Access the hidden states (e.g., the last layer)
        hidden_states = outputs.hidden_states[-1]  # Last layer hidden states

        # Access attention weights (if available in the model)
        # attention_weights = outputs.attentions  # For some models

        # ... use hidden_states and/or attention_weights in your logic ...
        # Returning the original outputs to avoid breaking the expected structure
        return outputs

# Replace your original model with the modified class
# The key change is to add attn_implementation="eager" to force the non-optimized approach
# Create a DistilBERT configuration object
config = DistilBertConfig.from_pretrained(model_name)  # Replace 'distilbert-base-uncased' with your model name if different

model = AttentionBERT.from_pretrained(model_name, config=config, attn_implementation="eager") # Now use the config object

In [14]:
from sklearn.metrics import accuracy_score

predictions = []
true_labels = []

for data in synthetic_dataset:
    inputs = tokenizer(data["text"], return_tensors="pt")
    outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits, dim=-1).item()

    predictions.append(predicted_label)
    true_labels.append(data["label"])

accuracy = accuracy_score(true_labels, predictions)
print("Accuracy after attention:", accuracy)

Accuracy after attention: 1.0
